**2DO**
1. Initialize DB
- initial database (from Sept'23)
- accumulate stats: N of posts by channel-day (will be beneficial for further analysis, e.g. % of news)
- update last_id
- embeddings
    - cut-off length of posts - summarization (2-3 sentences via sumy)
    - openai
    - pinecone
- analyze size - what's time span for pinecone limit of 100K (allow for more channels)

2. Script for auto-updates
- update as per last_ud
- update stats channel-day

3. Script to update pinecone
- set cut-ff date
- option1: combine w/step 2 - send new messages to pinecone (beforehand remove old ones)
- option2: load new messages from full DB and send to pinecone (beforehand remove old ones)

for today
- load 30 days - save locally as separate files
- prep cleaned_messages, summary (sumy) for long ones
- calc embeds openai
- combine into one file 
- send to pinecone 

# check env, set constants & imports

In [1]:
# Function to check env (colab or local)
import sys

def check_environment():
    if 'google.colab' in sys.modules:
        # Running in Google Colab
        return "Google Colab"

    if hasattr(sys, 'prefix'):
        if sys.prefix.startswith('/usr/local'):
            # Running in a Colab-like environment (could be Colab or another cloud environment)
            return "Colab-like environment"
        else:
            # Running in a local environment
            return "Local environment"

    # Default to local environment if checks are inconclusive
    return "Local environment"

In [2]:
# set paths and install packages depending on the environment

if check_environment() == 'Google Colab':
    !pip install telethon openai pinecone --quiet
    from google.colab import drive
    drive.mount('/content/drive')
    parent_path = '/content/drive/MyDrive/Colab Notebooks/Narratives/notebooks_data_ingest/'
    keys_path = '/content/drive/MyDrive/Colab Notebooks/Narratives/notebooks_data_ingest/keys'
    data_path = '/content/drive/MyDrive/Colab Notebooks/Narratives/TG_messages/'

else:
    parent_path = ''
    keys_path = 'keys/'
    data_path = '../TG_messages/'

In [3]:
from telethon import TelegramClient, events, sync
import pandas as pd
import numpy as np
import datetime
import json
import os
import re
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import pickle

import unicodedata

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
import nltk
from sumy.summarizers.lsa import LsaSummarizer

import openai
import pinecone

/Users/alexbadin/miniconda3/envs/db_prep/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
with open(keys_path+'api_keys.json') as f:
  data = json.loads(f.read())

# load TG credentials
api_id = data['api_id'] 
api_hash = data['api_hash']
phone = data['phone']

#load openai credentials
openai_key = data['openai_key']

## Initialize DB

In [5]:
# load channels
df_channels = pd.read_csv(parent_path+'channels.csv', sep=';')

In [6]:
def clean_text(text):
    # Unicode range for emojis
    emoji_pattern = re.compile("["
                               "\U0001F600-\U0001F64F"  # Emoticons
                               "\U0001F300-\U0001F5FF"  # Symbols & Pictographs
                               "\U0001F680-\U0001F6FF"  # Transport & Map Symbols
                               "\U0001F1E0-\U0001F1FF"  # Flags (iOS)
                               "]+", flags=re.UNICODE)
    
    # Remove emojis
    text = emoji_pattern.sub(r'', str(text))

    # Regular expression for URLs
    url_pattern = re.compile(r"http\S+|www\S+")
        
    # Remove URLs
    text = url_pattern.sub(r'', str(text))
    
    # Remove any remaining variation selectors
    text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')

    #Remove Foreign Agent text    
    pattern = re.compile(r'[А-ЯЁ18+]{3,}\s[А-ЯЁ()]{5,}[^\n]*ИНОСТРАННОГО АГЕНТА')
    text = pattern.sub('', text)
    name1 = 'ПИВОВАРОВА АЛЕКСЕЯ ВЛАДИМИРОВИЧА'
    text = text.replace(name1, '')

    return text

In [7]:
# summarize the news (select 2 most important sentences)
def summarize(text, language="russian", sentences_count=2):
    parser = PlaintextParser.from_string(text, Tokenizer(language))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentences_count)
    return ' '.join([str(sentence) for sentence in summary])

# NEED MORE FLEXIBLE MODEL
# summarize the news - need to keep length upto 750 characters

In [32]:
def process_new_messages(df, channel):    
    df = df.drop_duplicates(subset=['id']).copy() # create a copy of the DataFrame before modifying it
    df.loc[:, 'cleaned_message'] = df['message'].apply(clean_text) #remove emojis, urls, foreign agent text
    df = df[~df.cleaned_message.str.len().between(0, 30)] #remove empty or too short messages
    # summarize cleaned_messages: 2 sentences if length > 750, 3 sentences if length > 1500
    df.loc[:, 'summary'] = df['cleaned_message'].apply(lambda x: summarize(x, sentences_count=3) if len(x) > 750 else summarize(x, sentences_count=2) if len(x) > 500 else x)
    # add channel name & stance
    df.loc[:, 'channel'] = channel
    df.loc[:, 'stance'] = df['channel'].map(df_channels.set_index('channel_name')['stance'])
    return df

In [38]:
start_date = datetime.datetime(2023, 10, 1)
df_channel_stats = pd.DataFrame() # store N of posts per channel per day
session_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") # to name session stats file

# add new messages to channel df-file and update last_id in channels.csv
for i, row in tqdm_notebook(df_channels.iterrows(), total=df_channels.shape[0]):
    channel = row['channel_name']
    last_id = row['last_id']
    async with TelegramClient('session', api_id, api_hash) as client:
        # COLLECT NEW MESSAGES
        data = [] # for collecting new messages
        # check if last_id is integer (=set)
        try:
            offset_id = int(last_id)
        except:
            offset_id = 0
        async for message in client.iter_messages(channel, reverse=True, offset_date=start_date, offset_id=offset_id):
            data.append(message.to_dict())
        # if no new messages, skip
    print(f"Channel: {channel}, N of new messages: {len(data)}")
    if len(data) == 0:
        continue
    df = pd.DataFrame(data)
    
    # PROCESS NEW MESSAGES
    df = process_new_messages(df, channel)

    # APPEND TO MAIN DF (pickle to avoid problems with separators in csv)
    # if file exists, append to file
    if os.path.isfile(data_path+channel+'.pkl'):
        with open(data_path+channel+'.pkl', 'ab') as f:
            df_old = pd.read_pickle(data_path+channel+'.pkl')
            df = pd.concat([df_old, df], ignore_index=True)
            df.to_pickle(data_path+channel+'.pkl')
    else:
        df.to_pickle(data_path+channel+'.pkl')

    # save session stats for channel
    df_channel_stats[channel] = df['date'].dt.date.value_counts()
    # update last_id in channels.csv
    df_channels.loc[i, 'last_id'] = df['id'].max()

# save session stats for all channels & update channels.csv
df_channel_stats.to_csv(parent_path+f'session_stats/channel_stats_{session_time}.csv', sep=';', index=True)
df_channels.to_csv(parent_path+'channels.csv', sep=';', index=False)

  0%|          | 0/48 [00:00<?, ?it/s]

Channel: rt_russian, N of new messages: 2298


/var/folders/2f/4wrcxd5n7_zbgfnwpf77l3bm0000gn/T/ipykernel_50305/1432942270.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['summary'] = df['cleaned_message'].apply(lambda x: summarize(x, sentences_count=3) if len(x) > 750 else summarize(x, sentences_count=2) if len(x) > 500 else x)
/var/folders/2f/4wrcxd5n7_zbgfnwpf77l3bm0000gn/T/ipykernel_50305/1432942270.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['channel'] = channel
/var/folders/2f/4wrcxd5n7_zbgfnwpf77l3bm0000gn/T/ipykernel_50305/1432

Channel: ntvnews, N of new messages: 1002


/var/folders/2f/4wrcxd5n7_zbgfnwpf77l3bm0000gn/T/ipykernel_50305/1432942270.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['summary'] = df['cleaned_message'].apply(lambda x: summarize(x, sentences_count=3) if len(x) > 750 else summarize(x, sentences_count=2) if len(x) > 500 else x)
/var/folders/2f/4wrcxd5n7_zbgfnwpf77l3bm0000gn/T/ipykernel_50305/1432942270.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['channel'] = channel
/var/folders/2f/4wrcxd5n7_zbgfnwpf77l3bm0000gn/T/ipykernel_50305/1432

...

Channel: tvc_ru, N of new messages: 669


/var/folders/2f/4wrcxd5n7_zbgfnwpf77l3bm0000gn/T/ipykernel_50305/1432942270.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['summary'] = df['cleaned_message'].apply(lambda x: summarize(x, sentences_count=3) if len(x) > 750 else summarize(x, sentences_count=2) if len(x) > 500 else x)
/var/folders/2f/4wrcxd5n7_zbgfnwpf77l3bm0000gn/T/ipykernel_50305/1432942270.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['channel'] = channel
/var/folders/2f/4wrcxd5n7_zbgfnwpf77l3bm0000gn/T/ipykernel_50305/1432

Channel: strelkovii, N of new messages: 15


/var/folders/2f/4wrcxd5n7_zbgfnwpf77l3bm0000gn/T/ipykernel_50305/1432942270.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['summary'] = df['cleaned_message'].apply(lambda x: summarize(x, sentences_count=3) if len(x) > 750 else summarize(x, sentences_count=2) if len(x) > 500 else x)
/var/folders/2f/4wrcxd5n7_zbgfnwpf77l3bm0000gn/T/ipykernel_50305/1432942270.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['channel'] = channel
/var/folders/2f/4wrcxd5n7_zbgfnwpf77l3bm0000gn/T/ipykernel_50305/1432

Channel: concordgroup_official, N of new messages: 0


Need system for storing data that's flexible and allows for appending without loading
- CSV - problems with separators (can be partially resolved by replacing ';' with ','. Or using other separator)
- Pickle - does not allow appending; need to load old data each time to append new one
- Parquet - very caprisous - time formats, emojis, could not make it work

### Process downloaded data (if was not )

In [24]:
df_channels = pd.read_csv(parent_path+'channels.csv', sep=';')

In [42]:
# add summarization
# remove empty or very short messages (30-)

# list of files in data_path
files = os.listdir(data_path)
for file in tqdm_notebook(files):
    if file.endswith('.pkl'):
        df = pd.read_pickle(data_path+file)
        channel = file[:-4]
        # if df empty, skip
        if df.shape[0] == 0:
            continue
        df = process_new_messages(df, channel)
        df.to_pickle(data_path+file)

  0%|          | 0/48 [00:00<?, ?it/s]

Итак теперь имеем:
- channels.csv - список каналов с last_id & stance
- session_stats/channel_stats_{session_time}.csv - стат-ка по посл.скачиванию постов (постов по дням)
- набор pkl-файлов (df) с сырыми постами + cleaned_message + summary

## Embeddings

In [52]:
openai.api_key = openai_key

# function for openai embeddings
def get_embeddings(text, model="text-embedding-ada-002"):
    response = openai.Embedding.create(
        input=text,
        model=model
    )
    embeddings = response['data'][0]['embedding']
    return embeddings

In [53]:
# create embeddings for NEW messages, iterate over all files in data_path
files = os.listdir(data_path)
for file in tqdm_notebook(files):
    if file.endswith('.pkl'):
        df = pd.read_pickle(data_path+file)
        # if df empty, skip
        if df.shape[0] == 0:
            continue
        # check if embeddings already exist
        if 'embeddings' in df.columns:
            # calculate embeddings only where embeddings are not calculated yet
            df_temp = df[df['embeddings'].isnull()]
            df_temp.loc[:,'embeddings'] = df_temp['summary'].apply(get_embeddings)
            df.loc[df_temp.index, 'embeddings'] = df_temp['embeddings']
            
        else:
            df['embeddings'] = df['summary'].apply(get_embeddings)
        df.to_pickle(data_path+file)

  0%|          | 0/47 [00:00<?, ?it/s]

KeyboardInterrupt: 

### create embeddings in batches - same speed ()

No speed gain vs 1 by 1 embedding. Can also try: https://github.com/openai/openai-cookbook/blob/main/examples/api_request_parallel_processor.py

In [323]:
# function for openai embeddings in batches
def get_embeddings_batch(texts, model="text-embedding-ada-002"):
    embeddings = []
    for text in tqdm(texts):
        response = openai.Embedding.create(
            input=text,
            model=model
        )
        embeddings.append(response['data'][0]['embedding'])
    return embeddings

In [338]:
batch_size = 50
n_batches = int(np.ceil(df.shape[0]/batch_size))
for i in tqdm(range(n_batches)):
    start = i*batch_size
    end = start+batch_size
    res = get_embeddings_batch(df.loc[start:end, 'summary'])
    df.loc[start:end, 'embeddings'] = pd.Series(res)

100%|██████████| 8/8 [02:03<00:00, 15.44s/it]


In [ ]:
# calculate embeddings in batches
batch_size = 4

files = os.listdir(data_path)
for file in tqdm(files):
    if file.endswith('.pkl'):
        df = pd.read_pickle(data_path+file)
        # if df empty, skip
        if df.shape[0] == 0:
            continue
        # if embeddings already calculated, skip
        if 'embeddings' in df.columns:
            continue
        # calculate embeddings in batches
        n_batches = int(np.ceil(df.shape[0]/batch_size))
        df['embeddings'] = np.nan
        for i in tqdm(range(n_batches)):
            start = i*batch_size
            end = start+batch_size
            df.loc[start:end, 'embeddings'] = get_embeddings_batch(df.loc[start:end, 'summary'])
        df.to_pickle(data_path+file)

## Pinecone

### prepare dataset for pinecone 

In [76]:
# data from all channels for last 30 days. Includes only embeddings, id and metadata: channel, date, message, cleaned_message, summary

# set start date
start_date = pd.Timestamp(datetime.datetime.now() - datetime.timedelta(days=30), tz='UTC')

# list of files in data_path
files = os.listdir(data_path)
df_list = []
for file in tqdm_notebook(files):
    if file.endswith('.pkl'):
        df_temp = pd.read_pickle(data_path+file)
        # if df empty, skip
        if df_temp.shape[0] == 0:
            continue
        # select data for last 30 days
        df_temp = df_temp[df_temp['date'] >= start_date]
        # id as channel_id + message_id
        df_temp['id'] = df_temp['channel'] + '_' + df_temp['id'].astype(str)
        df_temp = df_temp[['id', 'channel','stance', 'date', 'message', 'views', 'cleaned_message', 'summary', 'embeddings']]
        df_list.append(df_temp)

  0%|          | 0/47 [00:00<?, ?it/s]

In [84]:
df4pine = pd.concat(df_list)
df4pine.shape, df4pine.date.min(), df4pine.date.max()

((16243, 9),
 Timestamp('2023-10-09 00:00:01+0000', tz='UTC'),
 Timestamp('2023-10-14 18:01:46+0000', tz='UTC'))

In [58]:
df4pine.reset_index(inplace=True, drop=True)

### send to pinecone

In [45]:
with open(keys_path+'api_keys.json') as f:
  data = json.loads(f.read())

# load pinecone credentials
pine_key = data['pine_key']
pine_env = data['pine_env']

In [61]:
# create pinecone index & upsert texts & embeddings

emb_size = len(df4pine.embeddings.iloc[0])

pinecone.init(api_key=pine_key, environment=pine_env)
index_name = 'tg-news'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=emb_size)

In [83]:
index_name = 'tg-news'
index = pinecone.Index(index_name)
batch_size = 10  # process everything in batches of 32

for i in tqdm_notebook(range(0, df4pine.shape[0], batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, df4pine.shape[0])
    # get batch of IDs, embeds and metadata
    ids_batch = df4pine['id'][i: i+batch_size]
    embeds_batch = df4pine['embeddings'][i: i+batch_size]
    # prep metadata: 'channel', 'date', 'message', 'views', 'cleaned_message', 'summary'
    meta_cols = ['channel', 'stance', 'date', 'message', 'views', 'cleaned_message', 'summary']
    meta_batch = df4pine[meta_cols][i: i+batch_size].to_dict(orient='records')
    
    # upsert to Pinecone 
    # !!! meta skipped due to errors
    to_upsert = zip(ids_batch, embeds_batch, meta_batch)
    index.upsert(vectors=list(to_upsert))

  0%|          | 0/1625 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [46]:
pinecone.init(api_key=pine_key, environment=pine_env)
index_name = 'tg-news'

index = pinecone.Index(index_name)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.17342,
 'namespaces': {'': {'vector_count': 17342}},
 'total_vector_count': 17342}

In [47]:
pinecone.describe_index(index_name)

IndexDescription(name='tg-news', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

### ask_media test

In [49]:
question = "Что происходит на украинском фронте?"
# get embeddings for question
question_emb = get_embeddings(question)

In [50]:
res = index.query(question_emb, top_k=10, include_metadata=True, filter=None)

In [51]:
res

{'matches': [{'id': 'tsargradtv_59556',
              'metadata': {},
              'score': 0.89841187,
              'values': []},
             {'id': 'voenacher_54247',
              'metadata': {},
              'score': 0.896420717,
              'values': []},
             {'id': 'tsargradtv_59454',
              'metadata': {},
              'score': 0.89340055,
              'values': []},
             {'id': 'tsargradtv_59340',
              'metadata': {},
              'score': 0.89340055,
              'values': []},
             {'id': 'tsargradtv_59228',
              'metadata': {},
              'score': 0.89340055,
              'values': []},
             {'id': 'tsargradtv_59139',
              'metadata': {},
              'score': 0.89340055,
              'values': []},
             {'id': 'bbcrussian_53612',
              'metadata': {'channel': 'bbcrussian',
                           'cleaned_message': 'На фронтах без существенных '
                           

# Updating DB's (local & pinecone)

- download message

In [ ]:
# download latest messages from all channels
df_channels = pd.read_csv(parent_path+'channels.csv', sep=';')

df_channel_stats = pd.DataFrame() # store N of posts per channel per day
session_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") # to name session stats file

# add new messages to channel's full csv-file and update last_id in channels.csv
for i, row in tqdm(df_channels.iterrows(), total=df_channels.shape[0]):
    channel = row['channel_name']
    last_id = row['last_id']
    async with TelegramClient('session', api_id, api_hash) as client:
        data = []
        # check if last_id is integer (=set)
        try:
            offset_id = int(last_id)
        except:
            offset_id = 0
        async for message in client.iter_messages(channel, reverse=True, offset_id=offset_id):
            data.append(message.to_dict())
        # if no new messages, skip
        if len(data) == 0:
            continue
        df = pd.DataFrame(data)
        # cleaning messages
        df.drop_duplicates(subset=['id'], inplace=True) #duplicates
        df = df[~df.cleaned_message.str.len().between(0, 30)] #empty or too short messages
        df['cleaned_message'] = df['message'].apply(clean_text) #remove emojis, urls, foreign agent text
        # summarize cleaned_messages: 2 sentences if length > 750, 3 sentences if length > 1500
        df['summary'] = df['cleaned_message'].apply(lambda x: summarize(x, sentences_count=2) if len(x) > 750 else summarize(x, sentences_count=3) if len(x) > 1500 else x)

        # save to pickle to avoid problems with separators in csv
        # if file exists, append to file
        if os.path.isfile(data_path+channel+'.pkl'):
            with open(data_path+channel+'.pkl', 'ab') as f:
                df.to_pickle(f)
        else:
            df.to_pickle(data_path+channel+'.pkl')

        # save session stats for channel
        df_channel_stats[channel] = df['date'].dt.date.value_counts()
        df_channel_stats.to_csv(parent_path+f'session_stats/channel_stats_{session_time}.csv', sep=';', index=True)
        # update last_id in channels.csv
        df_channels.loc[i, 'last_id'] = df['id'].max()
        df_channels.to_csv(parent_path+'channels.csv', sep=';', index=False)

In [ ]:
start_date = datetime.datetime(2023, 10, 9)
df_channel_stats = pd.DataFrame() # store N of posts per channel per day
session_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") # to name session stats file

# add new messages to channel's full csv-file and update last_id in channels.csv
for i, row in tqdm(df_channels.iterrows(), total=df_channels.shape[0]):
    channel = row['channel_name']
    last_id = row['last_id']
    async with TelegramClient('session', api_id, api_hash) as client:
        data = []
        # check if last_id is integer (=set)
        try:
            offset_id = int(last_id)
        except:
            offset_id = 0
        async for message in client.iter_messages(channel, reverse=True, offset_date=start_date, offset_id=offset_id):
            data.append(message.to_dict())
        # if no new messages, skip
        if len(data) == 0:
            continue
        df = pd.DataFrame(data)
        # cleaning messages
        df.drop_duplicates(subset=['id'], inplace=True) #duplicates
        df = df[~df.cleaned_message.str.len().between(0, 30)] #empty or too short messages
        df['cleaned_message'] = df['message'].apply(clean_text) #remove emojis, urls, foreign agent text
        # summarize cleaned_messages: 2 sentences if length > 750, 3 sentences if length > 1500
        df['summary'] = df['cleaned_message'].apply(lambda x: summarize(x, sentences_count=2) if len(x) > 750 else summarize(x, sentences_count=3) if len(x) > 1500 else x)

        # save to pickle to avoid problems with separators in csv
        # if file exists, append to file
        if os.path.isfile(data_path+channel+'.pkl'):
            with open(data_path+channel+'.pkl', 'ab') as f:
                df.to_pickle(f)
        else:
            df.to_pickle(data_path+channel+'.pkl')

        # save session stats for channel
        df_channel_stats[channel] = df['date'].dt.date.value_counts()
        df_channel_stats.to_csv(parent_path+f'session_stats/channel_stats_{session_time}.csv', sep=';', index=True)
        # update last_id in channels.csv
        df_channels.loc[i, 'last_id'] = df['id'].max()
        df_channels.to_csv(parent_path+'channels.csv', sep=';', index=False)

100%|██████████| 48/48 [01:08<00:00,  1.44s/it]
